<a href="https://colab.research.google.com/github/BrennaLobb/STC510/blob/main/module5project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Parse, clean, and organize the Jeopardy! question data file to train a Naive Bayesian classifier.

In [98]:
import os
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk import download
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from string import punctuation
from datetime import datetime
import json
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from re import sub

In [ ]:
jeopardy_og_file = pd.read_json("jeopardy.json")
#jeopardy_og_file[:50]

In [183]:
q_dict_value = jeopardy_og_file[['question','value']]
q_list = list(jeopardy_og_file['question'])

##this above line takes the questions column and money value and assigns it to it's own dictionary

In [166]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [167]:
# below gives us the sentences tokenized and results in a list of lists (1 list per each sentence)
sent_list = []
for x in q_list:
  sent_list.append(sent_tokenize(x.lower()))

In [168]:
English_stopwords = (stopwords.words('english') + list(punctuation))
stop_sent_list = []
for x in sent_list:
  if not x in English_stopwords:
    stop_sent_list.append(x)
#[x for x in sent_list if not x in English_stopwords]

In [169]:
#NEED TO STEM EACH QUESTION, NOT THE WORDS!!!
#now for stemming the sentences
stemmer = PorterStemmer()
stemmed_qlist = []
stem_q = []
for x in stop_sent_list:
  stemmed_qlist.append(stemmer.stem(str(x)))

In [182]:
combo_dict = pd.DataFrame([value_list, stemmed_qlist])
df = combo_dict.T
df.columns = ['money_value','question']
#combo dict now has the money values (as keys) assigned to their cleaned/parsed questions
#need to transpose the dataframe, and assign column names after

In [198]:
# need to change money values into integers (no commas and no dollar signs)
df['money_value'].loc[df['money_value'] == "$200"] = "low"
df['money_value'].loc[df['money_value'] == "$100"] = "low"
df['money_value'].loc[df['money_value'] == "$300"] = "low"
df['money_value'].loc[df['money_value'] == "$400"] = "low"
df['money_value'].loc[df['money_value'] == "$500"] = "low"
df['money_value'].loc[df['money_value'] == "$600"] = "low"
df['money_value'].loc[df['money_value'] == "$700"] = "low"
df['money_value'].loc[df['money_value'] == "$800"] = "low"
df['money_value'].loc[df['money_value'] == "$1000"] = "high"
df['money_value'].loc[df['money_value'] == "$1200"] = "high"
df['money_value'].loc[df['money_value'] == "$1400"] = "high"
df['money_value'].loc[df['money_value'] == "$1600"] = "high"
df['money_value'].loc[df['money_value'] == "$1800"] = "high"
df['money_value'].loc[df['money_value'] == "$2000"] = "high"

In [214]:
df = df.dropna()

In [216]:
# create a binary classifier ("high value" and "low value," based on the points available for each) for questions
#"high value" is for high cost questions, more money to do it right
#"low value" is for cheaper questions

X_train, X_test, y_train, y_test = train_test_split(df.question, 
                                                    df.money_value, random_state=1)
Tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = Tfidf_vectorizer.fit_transform(X_train)
X_test_tf = Tfidf_vectorizer.transform(X_test)

In [217]:
#now to run them through naive Bayes:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)
print("accuracy: ",accuracy_score(y_test,predictions))

accuracy:  0.7129435151151452
